In [1]:
import os
import sys
while not os.getcwd().endswith('ml'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())

In [14]:
import pandas as pd
import numpy as np
import copy

from datetime import datetime, timedelta

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#### Считывание данных

In [3]:
key = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/key.csv")
train = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/train.csv")
test = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/test.csv")
weather = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/weather.csv")

#### Описание данных

#### Предобработка данных

In [37]:
def minutes_from_midnight(time_str):
    if len(time_str) == 1:
        return np.nan
    return int(time_str[:2])*60 + int(time_str[2:])

In [4]:
def preprocessing(data, key, weather, target_field='units'):
    data = data.join(key.set_index("store_nbr"), on="store_nbr")
    data = data.join(weather.set_index(["station_nbr", "date"]), on=["station_nbr", "date"])
    
    data['sunrise'] = data['sunrise'].apply(minutes_from_midnight)
    data['sunrise'] = data['sunset'].apply(minutes_from_midnight)
#     data.drop(columns=["codesum"])
    data.fillna(-1, inplace=True)
    
    needed_columns = ["store_nbr", "item_nbr"]
    if target_field in data.columns:
        needed_columns.append(target_field)
    
    data = data[needed_columns]
    data = pd.get_dummies(data, columns=["store_nbr", "item_nbr"])
    if target_field in data.columns:
        data['units'] = np.log(data['units'] + 1)
        
    return data

In [6]:
preprocessed_train = preprocessing(train, key, weather)
preprocessed_test = preprocessing(test, key, weather)

#### Обучение базовой модели

In [252]:
y = preprocessed_train['units']
X = preprocessed_train.loc[:, preprocessed_train.columns != 'units']

In [253]:
model = RandomForestRegressor(criterion='mse')

In [254]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

/Users/skrrydg/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
model.fit(X_train, y_train)

/Users/skrrydg/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
y_predict = model.predict(X_test)

In [ ]:
def get_score(y_true, y_predict):
    return mean_squared_error(y_true, y_predict)

In [ ]:
get_score(y_test, y_predict)

In [248]:
print(y_predict[np.abs(y_predict - y_test) > 0.1])
print(y_test[np.abs(y_predict - y_test) > 0.1])

[6.20457996]
1488312    6.359574
Name: units, dtype: float64


In [231]:
model.score(X_test, y_test)

0.9999999335966548